# Import

In [2]:
!pip install pyDOE

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import tensorflow as tf
import datetime, os
#hide tf logs 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'} 
#0 (default) shows all, 1 to filter out INFO logs, 2 to additionally filter out WARNING logs, and 3 to additionally filter out ERROR logs
import scipy.optimize
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable 
from mpl_toolkits.mplot3d import Axes3D
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import seaborn as sns 
import codecs, json
import math
# generates same random numbers each time
np.random.seed(1234)
tf.random.set_seed(1234)
import random
import datetime
import shutil
print("TensorFlow version: {}".format(tf.__version__))
import random
from importlib import reload
import PINN  # python files (classes)
import pre_post
from pre_post import *
from PINN import *

2023-05-26 18:05:08.194851: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-26 18:05:08.291635: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-26 18:05:08.291648: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-26 18:05:11.522397: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

TensorFlow version: 2.11.0


# Resolution of a Multi-Phase-Field problem using OPEN-PINN 


## Theory

A general free energy description separating different physical phenomena, interfacial $f^{\text {intf }}$, chemical $f^{\text {chem }}$ and elastic energy $f^{\text {elast }}$ 

$F=\int_{\Omega} f^{\text {intf }}+f^{\text {chem }}+f^{\text {elast }}$



other contributions like magnetic and electric energy may be added in future applications.

The other contributions could be added added to the MPF free energy forumaltuion, such as the mechanical ($f^{chem}$) and magnetic ones.

\begin{aligned}
f^{\mathrm{intf}} & =\sum_{\alpha, \beta=1, \ldots, N, \alpha \neq \beta} \frac{4 \sigma_{\alpha \beta}}{\eta_{\alpha \beta}}\left\{-\frac{\eta_{\alpha \beta}^2}{\pi^2} \nabla \phi_\alpha \cdot \nabla \phi_\beta+\phi_\alpha \phi_\beta\right\}, \\
f^{\mathrm{chem}} & =\sum_{\alpha=1, \ldots, N} h\left(\phi_\alpha\right) f_\alpha\left(c_\alpha^i\right)+\tilde{\mu}^i\left(c^i-\sum_{\alpha=1, \ldots, N} \phi_\alpha c_\alpha^i\right) \\
f^{\mathrm{elast}} & =\frac{1}{2}\left\{\sum_{\alpha=1, \ldots, N} h\left(\phi_\alpha\right)\left(\bar{\epsilon}_\alpha-\bar{\epsilon}_\alpha^*-c_\alpha^i \bar{\epsilon}_\alpha^i\right) \overline{\bar{C}}_\alpha\left(\bar{\epsilon}_\alpha-\bar{\epsilon}_\alpha^*-c_\alpha^j \bar{\epsilon}_\alpha^j\right)\right\}
\end{aligned}

$\alpha$ and $\beta$. It may be anisotropic with respect to the relative orientation between the phases. $\eta_{\alpha \beta}$ is the interface width.

The sum convention over double indices of the components $i . N=N(x)$ is the local number of phases and we have the sum constraint.

$$
\sum_{\alpha=1, . ., N} \phi_\alpha=1
$$
$\sigma_{\alpha \beta}$ is the energy of the interface between phase-or grain. 

$[\mu] = \frac{\text{L}^4}{\text{T}}E $   : $    {m}^4{J}^{-1}{s}^{-1}$

$[\sigma] = \frac{\text{E}}{\text{L}^2}$  : $    {J}^{}{m}^{-2}$

$[L] =  {m}^3{J}^{-1}{s}^{-1}$

The MPF equations are derived :

$$
\dot{\phi}_\alpha=-\sum_{\beta=1, . ., N} \frac{\pi^2}{8 \eta N} \mu_{\alpha \beta}\left(\frac{\delta F}{\delta \phi_\alpha}-\frac{\delta F}{\delta \phi_\beta}\right) .
$$

This is a superposition of dual phase changes between pairs of phases.
$\mu_{\alpha \beta}$ is defined individually for each pair of phases and can be treated in the thin interface limit replacing it by the effective mobility. 
Inserting the free energy from previous equations, we calculate explicitly:

$$
\begin{aligned}
\dot{\phi}_\alpha & =\sum_{\beta=1, . ., N} \frac{\mu_{\alpha \beta}}{N}\left\{\sum_{\gamma=1, . ., N}\left[\sigma_{\beta \gamma}-\sigma_{\alpha \gamma}\right] I_\gamma+\frac{\pi^2}{8 \eta} h^{\prime} \Delta g_{\alpha \beta}\right\}, \\
I_\gamma & =\nabla^2 \phi_\gamma+\frac{\pi^2}{\eta^2} \phi_\gamma .
\end{aligned}
$$


$I_\gamma$ is the generalized curvature term. For anisotropic interfacial energies the respective torque term has to be added . $\Delta g_{\alpha \beta}$ comprises the derivative of the chemical free energy and the elastic free energy with respect to the phase-field variables. There arises, however, a consistency problem that remains unsolved to date: how to formulate an appropriate contour function $h\left(\phi_\alpha\right)$ for multiple junctions. A thermodynamically consistent form is the unity $h\left(\phi_\alpha\right)=\phi_\alpha$ with $h^{\prime}=1$. However, this disturbs the traveling wave solution of the double obstacle potential.


A generalization of the contour function $h_{\mathrm{DO}}$ which is suitable for multiple junctions and does not violate the sum constraint $\sum_{\alpha=1, \ldots, N} h\left(\phi_\alpha\right)=1$ hardly seems possible. In most simulations using the MPF the so-called antisymmetric approximation, which resigns from thermodynamic consistency at the multiple junctions, is thus used.

 Ifweassume the same interface width and the same interfacial energies for all phases, it simplifies the MPF equation and allows for a more straightforward implementation. In this case, the MPF equation becomes:

$\dot{\phi}_\alpha = \sum_{\beta=1}^{N} \mu_{\alpha\beta}\left[\sigma(\phi_\beta - \phi_\alpha)\nabla^2\phi_\alpha + \frac{\pi^2}{2\eta^2}(\phi_\alpha - \phi_\beta) + \frac{\pi}{\eta}\sqrt{\phi_\alpha\phi_\beta}\Delta g_{\alpha\beta}\right] $

